## Block Matching Regisrtation

This notebook outputs transformaed pointclouds + alignmnet matrices + warped images. 
This notebut is run in WSL since it is using ants , which is not available for windows. 

For vizualisation, go to " NC_Control_View" notebook , which is run from windows ... 

In [79]:
import sys
# win:
# project_path = 'D:/Code/repos/gad1b-redistribution/src'
# wsl:
project_path = '/mnt/d/Code/repos/pwreg'
sys.path.insert(1, f'{project_path}/pwreg')

from core.core import *
from visualize.viewers import *

import pickle
import pandas as pd

import inspect
# print(inspect.getsource(Something))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
project_path

'/mnt/d/Code/repos/pwreg'

### Global image alignment 
( TODO : allow rigid ) 

In [28]:
mask_tp2 = {'zmin': 51,'zmax': 164 + 1}
mask_tp1 = {'zmin': 51,'zmax': 168 + 1}

fixed = Image([0.658, 0.23, 0.23], filename = f'{project_path}/data/raw/img/no_change_control/Image_1-20FM.ome.tif', mask = mask_tp2) # tp 2
moving = Image([0.658, 0.23, 0.23], filename = f'{project_path}/data/raw/img/no_change_control/Image_1-20FJ.ome.tif', mask = mask_tp1) # tp 1

print(f'shape\nFixed {fixed.shape},\nMoving {moving.shape}')
print(f'masks :\nFixed {fixed.mask},\nMoving {moving.mask}')

shape
Fixed (114, 1024, 1024),
Moving (118, 1024, 1024)
masks :
Fixed {'zmin': 51, 'zmax': 165, 'xmin': None, 'xmax': None, 'ymin': None, 'ymax': None},
Moving {'zmin': 51, 'zmax': 169, 'xmin': None, 'xmax': None, 'ymin': None, 'ymax': None}


In [29]:
# put the whole image size here
voxels_f = fixed.split([114,1024,1024],0)
voxels_m = moving.split([118,1024,1024],0)

In [30]:
pair = BlockPair(voxels_f[0],voxels_m[0])
pair.register(verbose = True) # block 2 (blc2) -to-> block 1 (blc1)

fixed : ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (114, 1024, 1024)
	 Spacing    : (0.658, 0.23, 0.23)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

moving : ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (118, 1024, 1024)
	 Spacing    : (0.658, 0.23, 0.23)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]



In [31]:
save_image = 1
if save_image: 
    tif.imsave(f'{project_path}/data/processed/registered/no_change_control/1-20FJ_to_1-20FM_affine_TS_resolution_ZYX.tif', pair.warp(), imagej=True)

pair.alignment['affine'].to_json(f'{project_path}/data/processed/registered/no_change_control/1-20FJ_to_1-20FM_affine_TS_resolution_ZYX.json')

In [33]:
# af = {'matrix': pair.alignment['affine'].matrix, 'center' : pair.alignment['affine'].center}
# with open(f'{project_path}/data/processed/registered/no_change_control/1-20FJ_to_1-20FM_affine.pkl', 'wb') as f:
#     pickle.dump(af, f)

In [34]:
# with open(f'{project_path}/data/processed/registered/no_change_control/1-20FJ_to_1-20FM_affine.pkl', 'rb') as f:
#     af = pickle.load(f)

{'matrix': array([[ 1.01343539e+00,  5.01786922e-03, -3.49930976e-02,
          0.00000000e+00],
        [-7.74445434e-04,  1.00188341e+00, -1.23891263e-02,
          0.00000000e+00],
        [ 3.71186197e-04,  1.20298269e-02,  1.00269345e+00,
          0.00000000e+00],
        [-1.54117430e-01, -2.22441956e+00,  3.30081141e+00,
          1.00000000e+00]]),
 'center': array([0., 0., 0.])}

### Global points alignment

In [ ]:
segmentation_folder = '/mnt/d/Code/repos/psd95_segmentation/models/two_small_cnns/predictions/Gad1b_2021/Sequential_2tp_control/'
ml_seg1 = pd.read_csv(f'{segmentation_folder}ML_segmentation_1-20GA_12-22-2021_2CNN_gui.csv') # tp1
ml_seg2 = pd.read_csv(f'{segmentation_folder}ML_segmentation_1-20G6_12-22-2021_2CNN_gui.csv') # tp2

prob_thr = 0.5
ptc1 = Points(ml_seg1[['Z','Y','X']][ml_seg1["prob"]>prob_thr].to_numpy(), units='pix', resolution=[0.658, 0.23, 0.23])
ptc2 = Points(ml_seg2[['Z','Y','X']][ml_seg2["prob"]>prob_thr].to_numpy(), units='pix', resolution=[0.658, 0.23, 0.23])

In [ ]:
ptc1 = ptc1.crop({'zmin': 51,'zmax': 164 + 1}, units='pix')
ptc1 = ptc1.recenter([51,0,0], units = 'pix')

ptc2 = ptc2.crop({'zmin': 51,'zmax': 168 + 1}, units='pix')
ptc2 = ptc2.recenter([51,0,0], units = 'pix')

In [ ]:
ptc1.to_json(f'{project_path}/data/processed/registered/no_change_control/ptc1_global.json')
ptc2.to_json(f'{project_path}/data/processed/registered/no_change_control/ptc2_global.json')

In [ ]:
af = AffineTransform.from_json(f'{project_path}/data/processed/registered/no_change_control/1-20FJ_to_1-20FM_affine_TS_resolution_ZYX.json')
    
ptc1_tf = ptc1.transform(af, units='phs')
ptc1_tf.to_json(f'{project_path}/data/processed/registered/no_change_control/ptc1_to_ptc2_global.json')

### Block affine registration

In [39]:
# use the pre-registered 
prereg = Image([0.658, 0.23, 0.23],img = pair.warp())

Let's try with the overlap:

In [41]:
voxels_f = fixed.split([18,102,102],[2,5,5])
voxels_m = prereg.split([18,102,102],[2,5,5])

/mnt/d/Code/repos/pwreg/pwreg/core/core.py:102: UserWarning:

Specified voxel size + overlap don't cover the whole image.Image size is [ 114 1024 1024], block size [ 18 102 102], overlap [2 5 5] results in [ 7.         10.50515464 10.50515464] blocks.
Leaving some image out. 



In [43]:
pairs = [BlockPair(voxf,voxm) for voxf,voxm in zip(voxels_f,voxels_m)]
for pair in pairs: 
    pair.register()

In [44]:
BlockPairView(pairs, 10).write_volume(f'{project_path}/data/processed/registered/no_change_control/pw_registration_moving_1-20FJ_overlap_NN_ZYX.tif')
BlockView(voxels_f,10).write_volume(f'{project_path}/data/processed/registered/no_change_control/pw_registration_fixed_1-20FM_overlap_NN_ZYX.tif')

### Block points registration

In [74]:
segmentation_folder = '/mnt/d/Code/repos/psd95_segmentation/models/two_small_cnns/predictions/Gad1b_2021/Sequential_2tp_control/'
ptc2 = Points.from_predictions(f'{segmentation_folder}ML_segmentation_1-20G6_12-22-2021_2CNN_gui.csv', prob_thr=0.5, units='pix', resolution=[0.658, 0.23, 0.23]) # tp2

ptc2 = ptc2.crop(mask_tp2, units='pix')
ptc2 = ptc2.recenter([51,0,0], units = 'pix')

In [82]:
ptc2_in_blocks = ptc2.split(voxels_f)
PointsBlockView(ptc2_in_blocks, voxels_f, 10).to_json(f'{project_path}/data/processed/registered/no_change_control/ptc2_block_ZYX.json')

In [90]:
ptc1_global = Points.from_json(f'{project_path}/data/processed/registered/no_change_control/ptc1_global_tansformed_to_ptc2_TS_resolution_ZYX.json')
ptc1_to_ptc2_in_blocks = ptc1_global.pw_transform(pairs)
PointsBlockView(ptc1_to_ptc2_in_blocks, voxels_f, 10).to_json(f'{project_path}/data/processed/registered/no_change_control/ptc1_to_ptc2_block_ZYX.json')